In [3]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']
play_file_infixes = ['othello']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)
# aligned_othello_style1 = "data/shakespeare/data/align/plays/merged/othello_modern.snt.aligned"
# aligned_othello_style2 = "data/shakespeare/data/align/plays/merged/othello_original.snt.aligned"

# aligned_ant_cleo_style1 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_modern.snt.aligned"
# aligned_ant_cleo_style2 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_original.snt.aligned"

# aligned_asyoulikeit_style1 = "data/shakespeare/data/align/plays/merged/asyoulikeit_modern.snt.aligned"
# aligned_asyoulikeit_style2 = "data/shakespeare/data/align/plays/merged/asyoulikeit_original.snt.aligned"

# aligned_errors_style1 = "data/shakespeare/data/align/plays/merged/errors_modern.snt.aligned"
# aligned_errors_style2 = "data/shakespeare/data/align/plays/merged/errors_original.snt.aligned"

# aligned_hamlet_style1 = "data/shakespeare/data/align/plays/merged/hamlet_modern.snt.aligned"
# aligned_hamlet_style2 = "data/shakespeare/data/align/plays/merged/hamlet_original.snt.aligned"

# aligned_henryv_style1 = "data/shakespeare/data/align/plays/merged/henryv_modern.snt.aligned"
# aligned_henryv_style2 = "data/shakespeare/data/align/plays/merged/henryv_original.snt.aligned"

# aligned_juliuscaesar_style1 = "data/shakespeare/data/align/plays/merged/juliuscaesar_modern.snt.aligned"
# aligned_juliuscaesar_style2 = "data/shakespeare/data/align/plays/merged/juliuscaesar_original.snt.aligned"

# style1_tuples = unidecode.unidecode(open(aligned_style1).read())
# style2_tuples = unidecode.unidecode(open(aligned_style2).read())

# style1_tuples = style1_tuples.split("\n")
# style2_tuples = style2_tuples.split("\n")

In [4]:
# print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
# print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
# print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
# print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


In [5]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['1' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['0' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)
raw.shape

idx_col = [str(x) for x in range(len(raw))]
idx_col = np.array(idx_col).reshape(-1,1)
print(idx_col.shape)
print(raw.shape)
raw = np.hstack((idx_col, raw))
raw.shape

(3710, 1)
(3710, 2)


(3710, 3)

In [6]:
raw[2000]

array(['2000', "CASSIO  She's awake, sir.", '0'], dtype='<U497')

In [34]:
from sklearn.feature_extraction import DictVectorizer
import spacy
import scipy.sparse
nlp = spacy.load('en_core_web_sm')
print

vectorizer = DictVectorizer()
features = []

# feature engineering
    # construct index column
idx_col = np.array([raw[:,0]])
idx_col = idx_col.astype('f', copy=True)

#sent_len_feature = np.array([-1 for x in range(len(raw))])
raw_text_col = 1
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    feature_dict = {}
    pos_dict = {'NOUN_count': 0, 'ADV_count': 0, 'VERB_count': 0, 
                'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0}
    txt = str(raw[row_idx][raw_text_col])
    
    feature_dict['sent_len'] = len(txt)
    doc = nlp(txt)
    for token in doc:
        pos = token.pos_
        pos_dict[pos+'_count'] = pos_dict.get(pos+'_count', 0) + 1

    if pos_dict['NOUN_count'] == 0 or pos_dict['ADJ_count'] == 0:
        pos_dict['adj_noun_ratio'] = 0
    else:
        pos_dict['adj_noun_ratio'] = pos_dict['NOUN_count'] / pos_dict['ADJ_count'] 

    if pos_dict['VERB_count'] == 0 or pos_dict['ADV_count'] == 0:
        pos_dict['adv_verb_ratio'] = 0
    else:
        pos_dict['adv_verb_ratio'] = pos_dict['VERB_count'] / pos_dict['ADV_count'] 
        
    feature_dict.update(pos_dict)
    
    if row_idx % 1000 == 0:
        print("{} gold: {} {}\n".format(txt, raw[row_idx][LABEL_COL], feature_dict) )
        
    features.append(feature_dict)

labels_col = np.array([raw[:,LABEL_COL].astype(float)])

X = vectorizer.fit_transform(features)
print(type(X))
print(X.shape)
print(idx_col.T.shape)
X = scipy.sparse.hstack(( idx_col.T, X, labels_col.T ))
X = X.todense()
# X = np.transpose(X)
# X = X.astype('f', copy=True)
# print(X[0:5])
# print(X[3700:3705])

What's the matter, lieutenant? gold: 1 {'sent_len': 30, 'NOUN_count': 3, 'ADV_count': 0, 'VERB_count': 1, 'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'DET_count': 1, 'PUNCT_count': 2}

Therefore, good Emilia, Give me my nightly wearing, and adieu. gold: 1 {'sent_len': 62, 'NOUN_count': 1, 'ADV_count': 2, 'VERB_count': 2, 'ADJ_count': 2, 'adv_verb_ratio': 1.0, 'adj_noun_ratio': 0.5, 'PUNCT_count': 4, 'PROPN_count': 1, 'PRON_count': 1, 'CCONJ_count': 1}

CASSIO  She's awake, sir. gold: 0 {'sent_len': 25, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 1, 'ADJ_count': 1, 'adv_verb_ratio': 0, 'adj_noun_ratio': 1.0, 'PROPN_count': 1, 'SPACE_count': 1, 'PRON_count': 1, 'PUNCT_count': 2}

Who's shouting? gold: 0 {'sent_len': 15, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 2, 'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'PUNCT_count': 1}

<class 'scipy.sparse.csr.csr_matrix'>
(3710, 18)
(3710, 1)


In [37]:
X[1:2]



matrix([[ 1.,  0.,  1.,  0.,  0.,  2.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,
          0.,  2.,  0.,  0.,  0., 42.,  1.]])

In [38]:
from sklearn.linear_model import LogisticRegression
import time
import math
###############
# training ####
###############
# test breakout: 3500, 210
train_test_split_delim = math.floor(len(X) * .8)
print("total rows: {} traintest split delim: {}".format(len(X), train_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim
last_feat_col = X.shape[1] -2
X = np.random.permutation(X)

X_train = X[0:train_test_split_delim,1:last_feat_col ]
#X_train = X_train.reshape(-1, 1) #TODO: REMOVE
Y_train = X[0:train_test_split_delim,LABEL_COL]

X_test = X[train_test_split_delim:,1:last_feat_col ]
#X_test = X_test.reshape(-1, 1)
Y_test = X[train_test_split_delim:,LABEL_COL]

logreg = LogisticRegression(random_state=123)
logreg.fit(X_train, Y_train)

np.set_printoptions(suppress=True)


total rows: 3710 traintest split delim: 2968


In [39]:
X.shape

(3710, 20)

In [40]:
# evaluation
preds = logreg.predict(X_test)
test_indices = X[train_test_split_delim:,0]
if len(preds) != len(test_indices):
    raise ValueError("unmatched arrays")
    
results = []
for (idx, pred) in zip(test_indices,preds):
    gold_label = int(raw[int(idx)][-1])
    pred_label = int(pred)
    #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
    if gold_label == pred_label:
          results.append(1)
    else:
        results.append(0)
    
results[:5]


[1, 0, 0, 1, 1]

In [41]:
test_len = len(preds)
accuracy = sum(preds)/float(test_len)
print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228


accuracy: 0.4609164420485175


In [ ]:
#x = np.zeros(3, dtype=[('row_id','int32'),('text', 'S'), ('label', 'int32')])
X.shape[1]

In [ ]:
raw[9000:9002]

In [13]:
a = np.array([[1, 2], [3, 4], [5,6]])
a

array([[1, 2],
       [3, 4],
       [5, 6]])

In [14]:
b = np.array( [[7,8,9]] )

In [15]:
print(a.shape)
print(b.T.shape)

concat = np.hstack((b.T,a))
concat

(3, 2)
(3, 1)


array([[7, 1, 2],
       [8, 3, 4],
       [9, 5, 6]])